Based on https://haystack.deepset.ai/tutorials/01_basic_qa_pipeline

# Set up Document Store

In [1]:
from haystack.document_stores import WeaviateDocumentStore

document_store = WeaviateDocumentStore(
    host="http://weaviate", index="document", similarity="dot_product"
)


/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ERROR:root:`libsndfile` not found, it's probably not installed. The node will most likely crash. Please install soundfile's dependencies (https://python-soundfile.readthedocs.io/en/latest/)
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/soundfile.py", line 151, in <module>
    raise OSError('sndfile library not found')
OSError: sndfile library not found

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/soundfile.py", line 178, in <module>
    _snd = _ffi.dlopen(_os.path.join(_path, '_soundfile_data', _packaged_libname))
OSError: cannot load library '/usr/local/lib/python3.9/site-packages/_soundfile_data/libsndfi

# Preprocess Documents

In [2]:
from haystack.utils import (
    clean_wiki_text,
    convert_files_to_docs,
    fetch_archive_from_http,
)


# Let's first fetch some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "data/tutorial1"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

docs = convert_files_to_docs(
    dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True
)

document_store.write_documents(docs)


  0%|          | 0/2497 [00:00<?, ?it/s]/usr/local/lib/python3.9/site-packages/weaviate/batch/crud_batch.py:750: RuntimeWarning: You are manually batching this means you are NOT using the client's built-in multi-threading. Setting `batch_size` in `client.batch.configure()` to an int value will enabled this. Also see: https://weaviate.io/developers/weaviate/current/restful-api-references/batch.html#example-request-1
  warnings.warn(
10000it [00:03, 3123.13it/s]            


# Initialize Retriever, Reader and Pipeline

Retriever:

In [3]:
from haystack.nodes import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1",
    model_format="sentence_transformers",
)

document_store.update_embeddings(retriever)


Batches: 100%|██████████| 74/74 [02:09<00:00,  1.75s/it]


Reader:

In [4]:
from haystack.nodes import TransformersReader

reader = TransformersReader(
    model_name_or_path="distilbert-base-uncased-distilled-squad",
    tokenizer="distilbert-base-uncased",
    use_gpu=-1,
)


Pipeline:

In [5]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)


# Ask a question

In [6]:
from haystack.utils import print_answers

prediction = pipe.run(
    query="Who is the father of Arya Stark?",
    params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}},
)


print_answers(prediction, details="minimum")


Batches: 100%|██████████| 1/1 [00:00<00:00, 85.14it/s]



Query: Who is the father of Arya Stark?
Answers:
[   {   'answer': 'Maisie Williams',
        'context': 'f sword fighting by Syrio Forel.\n'
                   'Arya is portrayed by English actress Maisie Williams in '
                   "HBO's Emmy-winning television adaptation of the novel "
                   "series, ''Ga"},
    {   'answer': 'Eddard and Catelyn Stark',
        'context': '\n'
                   '=== Robb Stark ===\n'
                   'Robb Stark is the oldest child of Eddard and Catelyn '
                   'Stark, and the heir to Winterfell. He is not a POV '
                   'character, but features '},
    {   'answer': 'Eddard and Catelyn Stark',
        'context': ' Arya Stark ===\n'
                   'Arya Stark is the third child and younger daughter of '
                   'Eddard and Catelyn Stark. She serves as a POV character '
                   "for 33 chapters throughout ''A Game of"},
    {   'answer': 'Eddard and Catelyn Stark',
        'con